<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/11APRIL2025_FineTuning_LLM_Meta_Llama_3_8B_for_MEDAL_EVALDATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://llama.meta.com/docs/how-to-guides/fine-tuning/


https://medium.com/@frankmorales_91352/fine-tuning-meta-llama-3-8b-with-medal-a-refined-approach-for-enhanced-medical-language-b924d226b09d

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

! pip install trl==0.8.6 -q

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [2]:
from huggingface_hub import login

login(
  token=access_token_write, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

In [3]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer

In [4]:
# set device
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
torch.__version__

In [ ]:
!python --version
!nvcc --version
!nvidia-smi

In [ ]:
### conversational format
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}

### instruction format
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/train_dataset.json", split="train")

In [10]:
dataset

Dataset({
    features: ['abstract_id', 'text', 'location', 'label'],
    num_rows: 3000000
})

In [ ]:
print(dataset[6]['text'])

In [ ]:
print(dataset[6]['label'])

In [ ]:
dataset_test = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")

In [14]:
dataset_test

Dataset({
    features: ['abstract_id', 'text', 'location', 'label'],
    num_rows: 1000000
})

In [15]:
import re

def dataset_to_finetuning(dataset,nrec):

    question=dataset['text']
    answer_entry=dataset['label']


    !rm -rf /content/question_answering.txt
    filename='/content/question_answering.txt'


  # python object to be appended
    for n in range(nrec):
        #print(answer[n])
        if answer_entry[n] == None:
           answer_entry[n] = 'Not possible to get or use'
        string = answer_entry[n]

        str_question=question[n]
        question_final= re.sub("\n", "", str_question)

        answer = re.sub("\[", "", str(string))
        answer = re.sub("\]", "", str(answer))
        answer0=answer[1:len(answer)-1]

        if len(answer0)==0:
            answer='Not possible to get or use'

        #print(answer0)


        text='<s>[INST] %s [/INST] %s </s>'%(question_final,answer0)
        del string,answer0
        #print(text)
        with open(filename, 'a') as f:
            f.write(text + '\n')
        f.close()

    text0 = load_dataset("text", data_files="/content/question_answering.txt", split="train")

    dataset_final_Question=dataset['text'][0:nrec]
    dataset_final_Answer=dataset['label'][0:nrec]
    dataset_final_text=text0[0:nrec]['text']

    import pandas as pd

    datasetF = pd.DataFrame() # Create an empty DataFrame
    datasetF['Question'] = dataset_final_Question
    datasetF['Answer'] = dataset_final_Answer
    datasetF['text'] = dataset_final_text


    return datasetF

In [ ]:
datasetF=dataset_to_finetuning(dataset,500000)

In [ ]:
datasetF['Question'][6]

In [ ]:
datasetF['Answer'][6]

In [ ]:
datasetF['text'][6]

In [ ]:
datasetF

In [ ]:
datasetF_test=dataset_to_finetuning(dataset_test,20000)

In [ ]:
datasetF_test

In [ ]:
datasetF_test['Question'][6]

In [ ]:
datasetF_test['Answer'][6]

In [ ]:
datasetF_test['text'][6]

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
#model_id = "mistralai/Mistral-7B-Instruct-v0.1" #01 march 2024 AND 10/03/2024

model_id = "meta-llama/Meta-Llama-3-8B" #22 MAY 2024

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id


In [ ]:
print(model)

In [28]:
from peft import LoraConfig

peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.05,
        r=128,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

In [43]:
from transformers import TrainingArguments

args = TrainingArguments(

    output_dir="/content/gdrive/MyDrive/model/11APRIL2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata",

    #num_train_epochs=3,                     # number of training epochs
    num_train_epochs=0.3,                    # number of training epochs for POC
    #num_train_epochs=1,
    per_device_train_batch_size=1,          # batch size per device during training
    #2
    gradient_accumulation_steps=8,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    #gradient_checkpointing_kwargs={"use_reentrant": True},
    optim="adamw_torch_fused",              # use fused adamw optimizer

    warmup_ratio=0.15,
    weight_decay=0.05,

    logging_steps=100,                       # log every 10 steps
    learning_rate=5e-5,                     # learning rate, based on QLoRA paper # i used in the first model

    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=1.0,                      # max gradient norm based on QLoRA paper


    #lr_scheduler_type="constant",           # use constant learning rate scheduler
    lr_scheduler_type="cosine",            # lr_scheduler_type="cosine" (Cosine Annealing Learning Rate)

    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    gradient_checkpointing_kwargs={"use_reentrant": True},

    load_best_model_at_end=True,
    logging_dir="/content/gdrive/MyDrive/model/11APRIL2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata/logs",

    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    metric_for_best_model = "loss",
)

In [33]:
!pip install trl -q

In [38]:
model.config.use_cache=False
model.gradient_checkpointing_enable() #enable gradient checkpoint
#torch.utils.checkpoint.checkpoint(use_reentrant=False)

https://huggingface.co/docs/trl/main/en/sft_trainer

In [ ]:
from trl import SFTTrainer
import datasets
#from datasets import load_metric

# Convert Pandas DataFrame to Hugging Face Dataset
datasetF_hf = datasets.Dataset.from_pandas(datasetF)
datasetF_hf_test = datasets.Dataset.from_pandas(datasetF_test)

#max_seq_length = 3072 # max sequence length for model and packing of the dataset
max_seq_length = 1024

# Explicitly set the device using torch.device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Add a padding token to your tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Or, add a new padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

#metric = load_metric("squad")  # Assuming you're evaluating on SQuAD dataset
#def compute_metrics(eval_pred):
#    logits, labels = eval_pred
#    predictions = logits.argmax(axis=-1)
#    return metric.compute(predictions=predictions, references=labels)

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=datasetF_hf,
    dataset_text_field="text", ### added for the summarization dataset
    eval_dataset=datasetF_hf_test,

    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    #processing_class=tokenizer,
    packing=True,

    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
    }
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py

In [ ]:
from transformers import EarlyStoppingCallback


trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=5))


# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save model
trainer.save_model()

In [46]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

## Test Model and run Inference

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

peft_model_id = "/content/gdrive/MyDrive/model/11APRIL2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata"


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  attn_implementation="flash_attention_2",
  quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Let’s load our test dataset try to generate an instruction.

In [ ]:
print(model.peft_config)

In [ ]:
from datasets import load_dataset
from random import randint


# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")
nrec= randint(0, len(eval_dataset))
nrec=6

# Test on sample
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map
prompt =  eval_dataset[nrec]['text']


outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [ ]:
ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
ganswer

In [51]:
print(f"Query:\n{eval_dataset[nrec]['text']}")
print()
oanswer=str(eval_dataset[nrec]['label'])
oanswer=oanswer[2:len(oanswer)-2]
print(f"Original Answer:\n{oanswer}")
print()
ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
qc=str(ganswer).find('[INST]')
ganswer=ganswer[0:qc-7]
qc0=str(ganswer).find('[INST]')
ganswer=str(ganswer)[0:qc0]
qc=str(ganswer).find('[/INST]')
if qc>0:
  ganswer=ganswer[qc+8:len(ganswer)]
print(f"Generated Answer:\n{ganswer}")
print()
if ganswer == oanswer:
  print("Match")
else:
  print("NO Match")

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

In [ ]:
nrec=6
print(f"Query:\n{eval_dataset[nrec]['text']}")
print()
print(f"Original Answer:\n{eval_dataset[nrec]['label']}")
print()
print(f"Full Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

In [ ]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
qc=str(ganswer).find('[/INST]')
#print(qc)
if int(qc)<0:
    #print(f"Generated Answer:\n{ganswer}")
    ganswer=outputs[0]['generated_text'][len(prompt)+8:].strip()
else:
    ganswer=ganswer[qc:len(ganswer)]
ganswer

In [ ]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
qc0=str(ganswer).find('[INST]')
ganswer=str(ganswer)[0:qc0-8]
qc=str(ganswer).find('[/INST]')

if qc>=0:
  ganswer=ganswer[qc+8:len(ganswer)]

print(f"Generated Answer:\n{ganswer}")

Generated Answer:
antral follicle count


In [ ]:
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
from transformers import pipeline
from datasets import load_dataset

# Load a pre-trained sentence embedding model
model_embedding = SentenceTransformer('all-mpnet-base-v2')

# Load your test dataset
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")

# Create the generation pipeline
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

def evaluate(sample):
    prompt = sample['text']
    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.7,
                                  top_k=50, top_p=0.7, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

    ganswer = outputs[0]['generated_text'][len(prompt):].strip()
    qc0 = str(ganswer).find('[/INST]')
    qc1 = str(ganswer).find('[INST]')
    ganswer = str(ganswer)[qc0 + 8:qc1 - 8]

    oanswer = str(sample['label'])
    oanswer = oanswer[2:len(oanswer) - 2]

    if len(oanswer) == 0:
        oanswer = 'Not possible to get or use'

    #print('\n\n')
    #print(f"Question: {prompt}")
    #print(f"Generated Answer: {ganswer}")
    #print(f"Original Answer: {oanswer}")

    # Generate sentence embeddings
    embedding1 = model_embedding.encode(ganswer, convert_to_tensor=True)
    embedding2 = model_embedding.encode(oanswer, convert_to_tensor=True)

    # Calculate cosine similarity
    cosine_sim = util.cos_sim(embedding1, embedding2).item()

    # Set a similarity threshold (adjust as needed)
    if cosine_sim >= 0.5:
        #print("Match")
        return 1
    else:
        return 0


success_rate = []
number_of_eval_samples = 10  # Adjust the number of samples as needed

# Evaluate the model
for n in tqdm(range(number_of_eval_samples)):
    s = eval_dataset[n]
    success_rate.append(evaluate(s))


In [79]:
# Calculate accuracy
accuracy = sum(success_rate) / len(success_rate)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6


## Tensorbard

In [ ]:
%rm -rf /content/tensorbard
%mkdir -p /content/tensorbard
%cd /content/tensorbard
!git lfs install
!git clone https://huggingface.co/frankmorales2020/11APRIL2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/tensorbard/11APRIL2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata/logs/